In [241]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

### Bidding Model  
At the Electricity Markets, Participants have to communicate increasing bidding curves of price-volume pairs before market clearing.  
Depending on the market clearing price, the delivery of each participant becomes known  
We approximate this relation by using linear interpolation of volumes and presetting price Points, for example based on probabilities.

$$
y_t = \frac{c_t - P_{t,i} }{P_{t,i+1} - P_{t,i}} \cdot x_{i,t} + \frac{P_{t,i+1} - c_t }{P_{t,i+1} - P{t,i}} \cdot x_{i+1,t}, \qquad \text{if} \; P_{t,i} \leq c_t \leq P_{t,i+1}
$$

The Volumes have to be in increasing order:

$$
x_{i,t} \leq x_{i+1,t}
$$

In [260]:
# Uncertainty Set of Prices and Inflows
Omega = [(price = 10.0, fill = 15), (price = 20.0, fill = 10), (price = 30.0, fill = 5)]
P = [1/length(Omega) for om in Omega]
# One Price Point around each uncertain Price
PPoints = [5.0, 15, 25, 35]

4-element Vector{Float64}:
  5.0
 15.0
 25.0
 35.0

## Bidding Model
### Version 1 - Inactive Constraints

We Bid a commodity as price taker with an increasing price curve. 

$$
\begin{align*}
    W &= \text{Maximum Inventory/Resource for own production} \\
    w &= \text{Production using own resource (for no cost)} \\
    b &= \text{Balncing / Buying commodity on market to fulfill delivery}
    x_i &= \text{Bid at price point} P_i \\
    y_i &= \text{Volume to deliver after market clearing}
\end{align*}
$$
The objective is our revenue generated from sales, minus the balancing we had to do to fulfill it
$$
\begin{equation*}
    \max \; \rightarrow \; c_s \cdot y - c_b \cdot b 
\end{equation*}
$$

Only the price points $i = 1,\ldots,I$ that encompass the uncertain price will be relevant. We deactivate the rest of the constrains by setting their coefficients to 0 and get the redundant constraint $0 = 0$ 

In [261]:
function subproblem_builder_inactive(subproblem::Model, node::Int64)
    @variable(subproblem, 0 <= x[i = 1:length(PPoints)] <= 50, SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= W <= 100, SDDP.State, initial_value = 80)
    @variable(subproblem, y[i = 1:length(PPoints)-1] >= 0)
    @variable(subproblem, w[i = 1:length(PPoints)-1] >= 0)
    @variable(subproblem, b[i = 1:length(PPoints)-1] >= 0)
    @variable(subproblem, f >= 0)
    @variable(subproblem, s >= 0)
    
    @constraint(subproblem, clearing[i = 1:length(PPoints)-1], y[i] == 1* x[i].in  +  1* x[i+1].in)
    @constraint(subproblem, increasing[i = 1:length(PPoints)-1], x[i].out <= x[i+1].out)
    if node == 1
        @stageobjective(subproblem, 0)
        @constraint(subproblem, transfer, W.in == W.out)
    else
        @constraint(subproblem, production[i = 1:length(PPoints)-1], y[i] == w[i] + b[i])
        @constraint(subproblem, balance, W.out == W.in - sum(w[i] for i in 1:length(PPoints)-1) + f - s)
        @stageobjective(subproblem, 0)
        SDDP.parameterize(subproblem, Omega, P) do om
            # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
            # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
            JuMP.fix(f, om.fill, force=true)
            for i in 1:length(PPoints)-1
                if (om.price >= PPoints[i]) && (om.price <= PPoints[i+1])
                    set_normalized_coefficient(clearing[i], y[i], 1)
                    set_normalized_coefficient(clearing[i], x[i].in, -((om.price - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                    set_normalized_coefficient(clearing[i], x[i+1].in, -((PPoints[i+1] - om.price)/(PPoints[i+1] - PPoints[i])))
                    set_normalized_coefficient(balance, w[i], 1)
                    set_normalized_coefficient(production[i], w[i], -1)
                    set_normalized_coefficient(production[i], b[i], -1)
                    set_normalized_coefficient(production[i], y[i], 1)
                    @stageobjective(subproblem, om.price * y[i] -  om.price * 1.2 * b[i])
                else
                    set_normalized_coefficient(balance, w[i], 0)
                    set_normalized_coefficient(production[i], w[i], 0)
                    set_normalized_coefficient(production[i], b[i], 0)
                    set_normalized_coefficient(production[i], y[i], 0)
                    set_normalized_coefficient(clearing[i], y[i], 0)
                    set_normalized_coefficient(clearing[i], x[i].in, 0)
                    set_normalized_coefficient(clearing[i], x[i+1].in, 0)
                end
            end
        end
    end
    return
end

subproblem_builder_inactive (generic function with 1 method)

In [262]:
model = SDDP.LinearPolicyGraph(
    subproblem_builder_inactive;
    stages = 7,
    sense = :Max,
    upper_bound = 1e5,
    optimizer = CPLEX.Optimizer
)

A policy graph with 7 nodes.
 Node indices: 1, 2, 3, 4, 5, 6, 7


In [263]:
det_equiv = SDDP.deterministic_equivalent(model, CPLEX.Optimizer)
JuMP.write_to_file(det_equiv, "det_equiv.lp")
optimize!(det_equiv)
solution_summary(det_equiv; verbose=true)


Dual simplex solved model.

CPLEX Error  3003: Not a mixed-integer problem.
CPLEX Error  3003: Not a mixed-integer problem.
CPLEX Error  3003: Not a mixed-integer problem.
CPLEX Error  3003: Not a mixed-integer problem.
CPLEX Error  3003: Not a mixed-integer problem.
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Parallel mode: deterministic, using up to 12 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 10 threads...
 * Starting primal Simplex on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 11290 rows and 18949 columns.
Aggregator did 5097 substitutions.
All rows and columns eliminated.
Presolve time = 0.02 sec. (10.01 ticks)


* Solver : CPLEX

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Result count       : 1
  Has duals          : true
  Message from the solver:
  "optimal"

* Candidate solution
  Objective value      : -1.20000e+03
  Objective bound      : -1.20000e+03
  Dual objective value : -1.20000e+03
  Primal solution :
    W_in#1 : 8.00000e+01
    W_in#2 : 8.00000e+01
    W_in#3 : 0.00000e+00
    W_in#4 : 0.00000e+00
    W_in#5 : 0.00000e+00
    W_in#6 : 0.00000e+00
    W_in#7 : 0.00000e+00
    W_out#1 : 8.00000e+01
    W_out#2 : 0.00000e+00
    W_out#3 : 0.00000e+00
    W_out#4 : 0.00000e+00
    W_out#5 : 0.00000e+00
    W_out#6 : 0.00000e+00
    W_out#7 : 0.00000e+00
    _[22]#1 : 1.00000e+05
    _[22]#2 : 1.00000e+05
    _[22]#3 : 1.00000e+05
    _[22]#4 : 1.00000e+05
    _[22]#5 : 1.00000e+05
    _[22]#6 : 1.00000e+05
    _[22]#7 : 0.00000e+00
    b[1]#1 : 0.00000e+00
    b[1]#2 : 0.00000e+00
    b[1]#3 : 0.00000e+00
    

In [264]:
SDDP.train(model; stopping_rules = [SDDP.BoundStalling(10, 1e-4), SDDP.IterationLimit(100)])

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 7
  state variables : 5
  scenarios       : 7.29000e+02
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [22, 22]
  VariableRef in MOI.EqualTo{Float64}     : [1, 1]
  VariableRef in MOI.LessThan{Float64}    : [6, 6]
  AffExpr in MOI.LessThan{Float64}        : [3, 3]
  VariableRef in MOI.GreaterThan{Float64} : [15, 16]
  AffExpr in MOI.EqualTo{Float64}         : [4, 7]
numerical stability report
  matrix range     [5e-01, 1e+00]
  objective range  [1e+00, 4e+01]
  bounds range     [5e+01, 1e+05]
  rhs range        [0e+00, 0e+00]
-------------------------------------------------------------------
 iteratio


         3   3.016753e+03  4.178004e+03  3.299999e-02        78   1
         4   3.719783e+03  3.488440e+03  4.300022e-02       104   1
         5   2.700000e+03  3.481480e+03  5.200005e-02       130   1
         6   3.145714e+03  3.459494e+03  6.100011e-02       156   1
         7   3.734157e+03  3.459164e+03  7.200003e-02       182   1
         8   3.153346e+03  3.438083e+03  8.200002e-02       208   1
         9   2.663677e+03  3.436920e+03  9.200001e-02       234   1
        10   3.433680e+03  3.431496e+03  1.020000e-01       260   1
        11   3.457931e+03  3.428243e+03  1.120000e-01       286   1
        12   3.412882e+03  3.417537e+03  1.220002e-01       312   1
        13

   3.601485e+03  3.416383e+03  1.320002e-01       338   1
        14   3.688123e+03  3.394741e+03  1.420002e-01       364   1
        15   2.708306e+03  3.388006e+03  1.530001e-01       390   1
        16   3.410017e+03  3.368151e+03  1.630001e-01       416   1
        17   2.662536e+03  3.366629e+03  1.740000e-01       442   1
        18   3.436913e+03  3.362030e+03  1.860001e-01       468   1
        19   3.248196e+03  3.362011e+03  1.970000e-01       494   1
        20   3.383125e+03  3.360595e+03  2.080002e-01       520   1
        21   1.880172e+03  3.360250e+03  2.170000e-01       546   1
        22   3.502044e+03  3.358203e+03  2.290001e-01       572   1


        23   3.540000e+03  3.358194e+03  2.390001e-01       598   1
        24   3.589319e+03  3.357615e+03  2.500000e-01       624   1
        25   3.694723e+03  3.356804e+03  2.600000e-01       650   1
        26   2.722321e+03  3.354297e+03  2.710001e-01       676   1
        27   3.490000e+03  3.354277e+03  2.830000e-01       702   1
        28   3.344456e+03  3.354051e+03  2.940001e-01       728   1
        29   3.162822e+03  3.353692e+03  3.050001e-01       754   1
        30   2.588293e+03  3.352119e+03  3.160000e-01       780   1
        31   2.750000e+03  3.352119e+03  3.280001e-01       806   1
 

       32   3.607179e+03  3.352115e+03  3.380001e-01       832   1
        33   2.985415e+03  3.352104e+03  3.480000e-01       858   1
        34   3.632109e+03  3.351935e+03  3.580000e-01       884   1
        35   2.991649e+03  3.350658e+03  3.680000e-01       910   1
        36   3.419541e+03  3.350257e+03  3.800001e-01       936   1
        37   3.311218e+03  3.348941e+03  3.900001e-01       962   1
        38   3.610452e+03  3.348918e+03  3.990002e-01       988   1
        39   2.699872e+03  3.348816e+03  4.110000e-01      1014   1
        40   2.488140e+03  3.348793e+03  4.220002e-01      1040   1
        41   3.749183e+03  3.348657e+03  4.320002e-01      1066   1


...

        68   3.522993e+03  3.344672e+03  7.580001e-01      1768   1
        69   3.529785e+03  3.344411e+03  7.750001e-01      1794   1
        70   3.700000e+03  3.344409e+03  7.910001e-01      1820   1
        71   3.025005e+03  3.344403e+03  8.040001e-01      1846   1
        72   3.800000e+03  3.344403e+03  8.190000e-01      1872   1
        73   3.242667e+03  3.344398e+03  8.340001e-01      1898   1
        74   2.457957e+03  3.344386e+03  8.490002e-01      1924   1


        75   2.974937e+03  3.344348e+03  8.620000e-01      1950   1
        76   3.893749e+03  3.344317e+03  8.750000e-01      1976   1
        77   3.099705e+03  3.344216e+03  8.900001e-01      2002   1
        78   3.350000e+03  3.344216e+03  9.020002e-01      2028   1
        79   3.499983e+03  3.344214e+03  9.120002e-01      2054   1
        80   3.550000e+03  3.344214e+03  9.240000e-01      2080   1
        81   3.648889e+03  3.344214e+03  9.360001e-01      2106   1
        82   2.949970e+03  3.344208e+03  9.500000e-01      2132   1
        83 

  3.694937e+03  3.344208e+03  9.620001e-01      2158   1
        84   3.350129e+03  3.344192e+03  9.750001e-01      2184   1
        85   3.595000e+03  3.344192e+03  9.860001e-01      2210   1
        86   3.549578e+03  3.344143e+03  9.980001e-01      2236   1
        87   3.125000e+03  3.344142e+03  1.011000e+00      2262   1
        88   2.827498e+03  3.344138e+03  1.024000e+00      2288   1
        89   3.839783e+03  3.344118e+03  1.038000e+00      2314   1
        90   3.779960e+03  3.344058e+03  1.053000e+00      2340   1


        91   3.650000e+03  3.344058e+03  1.065000e+00      2366   1
        92   3.700000e+03  3.344058e+03  1.078000e+00      2392   1
        93   3.469806e+03  3.344051e+03  1.091000e+00      2418   1
        94   4.019986e+03  3.344031e+03  1.107000e+00      2444   1
        95   3.395000e+03  3.344031e+03  1.118000e+00      2470   1
        96   3.870000e+03  3.344030e+03  1.139000e+00      2496   1
        97   3.000000e+03  3.344029e+03  1.160000e+00      2522   1


        98   3.440000e+03  3.344027e+03  1.172000e+00      2548   1
        99   3.699654e+03  3.343997e+03  1.183000e+00      2574   1
       100   3.400000e+03  3.343997e+03  1.195000e+00      2600   1
-------------------------------------------------------------------
status         : iteration_limit
total time (s) : 1.195000e+00
total solves   : 2600
best bound     :  3.343997e+03
simulation ci  :  3.257431e+03 ± 1.169133e+02
numeric issues : 0
-------------------------------------------------------------------



#### Decision Rule

In [265]:
rule = SDDP.DecisionRule(model; node = 1)
solution = SDDP.evaluate(
    rule;
    incoming_state = Dict(:W => 80.0),
    controls_to_record = [:y, :w, :b],
)

(stage_objective = 0, outgoing_state = Dict(Symbol("x[3]") => 50.0, Symbol("x[4]") => 50.0, :W => 80.0, Symbol("x[1]") => 0.0, Symbol("x[2]") => 0.0), controls = Dict(:w => [0.0, 0.0, 0.0], :b => [0.0, 0.0, 0.0], :y => [0.0, 0.0, 0.0]))

In [266]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model,
    # The number of replications.
    100,
    # A list of names to record the values of.
    [:x, :y, :w, :W, :b, :f],
)

100-element Vector{Vector{Dict{Symbol, Any}}}:
 [Dict(:b => [0.0, 0.0, 0.0], :w => [0.0, 0.0, 0.0], :bellman_term => 3343.9974793227475, :node_index => 1, :objective_state => nothing, :belief => Dict(1 => 1.0), :x => SDDP.State{Float64}[SDDP.State{Float64}(0.0, 0.0), SDDP.State{Float64}(0.0, 0.0), SDDP.State{Float64}(0.0, 50.0), SDDP.State{Float64}(0.0, 50.0)], :stage_objective => 0, :f => 0.0, :noise_term => nothing…), Dict(:b => [0.0, 0.0, 0.0], :w => [0.0, 0.0, 50.0], :bellman_term => 2018.5356478447875, :node_index => 2, :objective_state => nothing, :belief => Dict(2 => 1.0), :x => SDDP.State{Float64}[SDDP.State{Float64}(0.0, 0.0), SDDP.State{Float64}(0.0, 0.0), SDDP.State{Float64}(50.0, 30.32699098552794), SDDP.State{Float64}(50.0, 50.0)], :stage_objective => 1500.0, :f => 5.0, :noise_term => (price = 30.0, fill = 5)…), Dict(:b => [0.0, 0.0, 0.16349549276397113], :w => [0.0, 0.0, 40.0], :bellman_term => 862.5925925925887, :node_index => 3, :objective_state => nothing, :belief => D

In [267]:
replication = 28
stage = 7
println("Bidding Curve Values: ", simulations[replication][stage][:x])
println("Inventory: ", simulations[replication][stage][:W])
println("Cleared Volume: ", simulations[replication][stage][:y])
println("Own Producion: ", simulations[replication][stage][:w])
println("Purchases: ", simulations[replication][stage][:b])
println("Filling in that stage: ", simulations[replication][stage][:f])

Bidding Curve Values: SDDP.State{Float64}[SDDP.State{Float64}(32.49999999999978, 0.0), SDDP.State{Float64}(32.49999999999978, 0.0), SDDP.State{Float64}(32.49999999999977, 50.0), SDDP.State{Float64}(32.49999999999977, 50.0)]
Inventory: SDDP.State{Float64}(22.49999999999994, 0.0)
Cleared Volume: [0.0, 0.0, 32.49999999999977]
Own Producion: [0.0, 0.0, 27.49999999999994]
Purchases: [0.0, 0.0, 4.999999999999833]
Filling in that stage: 5.0


In [268]:
plt = SDDP.SpaghettiPlot(simulations)
display(plt)

A spaghetti plot with 100 scenarios and 7 stages.

In [269]:
print(model[1])

Node 1
  # State variables : 5
  # Children        : 1
  # Noise terms     : 1


In [270]:
SDDP.write_subproblem_to_file(model[7], "subproblem_7.mof.json")
sbproblem = JuMP.read_from_file("subproblem_7.mof.json")
println(sbproblem)

Max 20 y[2] - 24 b[2] + x22
Subject to
 clearing[1] : 0 == 0.0
 clearing[2] : -0.5 x[2]_in - 0.5 x[3]_in + y[2] == 0.0
 clearing[3] : 0 == 0.0
 production[1] : 0 == 0.0
 production[2] : y[2] - w[2] - b[2] == 0.0
 production[3] : 0 == 0.0
 balance : -W_in - f + W_out + w[2] + s == 0.0
 increasing[1] : x[1]_out - x[2]_out <= 0.0
 increasing[2] : x[2]_out - x[3]_out <= 0.0
 increasing[3] : x[3]_out - x[4]_out <= 0.0
 x[1]_in == 14.999999999999634
 x[2]_in == 14.999999999999634
 x[3]_in == 14.99999999999963
 x[4]_in == 14.99999999999963
 W_in == 5.0
 f == 10.0
 x[1]_out >= 0.0
 x[2]_out >= 0.0
 x[3]_out >= 0.0
 x[4]_out >= 0.0
 W_out >= 0.0
 y[1] >= 0.0
 y[2] >= 0.0
 y[3] >= 0.0
 w[1] >= 0.0
 w[2] >= 0.0
 w[3] >= 0.0
 b[1] >= 0.0
 b[2] >= 0.0
 b[3] >= 0.0
 s >= 0.0
 x22 >= 0.0
 x[1]_out <= 50.0
 x[2]_out <= 50.0
 x[3]_out <= 50.0
 x[4]_out <= 50.0
 W_out <= 100.0
 x22 <= 0.0



### Add Capacity of multiple Hours per Stage  
At the electricity market, we bid for every hour of the day, so $t = 1, \ldots, T$  
For every hour, we send price-volume pairs $i = 1, \ldots, I \; \rightarrow$ we have to determine $I \cdot T$ bids.  
This has multiple consequences:
* For every hour of the day, we have to obtain the delivery from market clearing
* The stageobjective is now the sum of all market deliveries and prices for each hour. This is difficult, as we can't just sum over all i's and t's, but have to select the cleared $i_t$ for every hour $t$.
* The inventory $W$ now changes corresponding to the sum of production of the day.  

Idea: In SDDP.parametrize(), find the following set:

$$
\begin{equation*}
\mathcal{I}_t = \{ i_t \; | \; P_{i,t} \leq c_t \leq P_{i+1,t}, \; t = 1,\ldots,T   \}
\end{equation*} 
$$
Then for all restrictions: 
* If $i \in \mathcal{I}_t$: Activate the constraints by setting the parameters accordingly
* Else: Activate the constraints

The corresponding constraint is
$$
y_{i_t,t} = \frac{c_t - P_{t,i} }{P_{t,i+1} - P_{t,i}} \cdot x_{i,t} + \frac{P_{t,i+1} - c_t }{P_{t,i+1} - P{t,i}} \cdot x_{i+1,t}, \qquad \text{if} \; P_{t,i} \leq c_t \leq P_{t,i+1}
$$
For every $i \neq i_t$ the constraint has to be deactivated, f.e. by setting every coefficient to zero to obtain $0=0$.

For the objective: At every node,
$$
\max \; \rightarrow \; \sum\limits_{t = 1}^T c_t \cdot y_{i_t, t} - \mu \cdot b_t
$$
What we do is:
* Find the set $\mathcal{I}_t$
* Define the stageobjective by summing over $\mathcal{I}_t$.

In [284]:
# Time Window per Stage
T = 24
# Uncertainty Set of Prices and Inflows
Omega_t = [(price = [10.0 for t in 1:T], fill = 15), (price = [20.0 for t in 1:T], fill = 10), (price = [30.0 for t in 1:T], fill = 5)]
P_t = [1/length(Omega) for om in Omega]
# One Price Point around each uncertain Price
PPoints = [5.0, 15, 25, 35]

4-element Vector{Float64}:
  5.0
 15.0
 25.0
 35.0

In [294]:
function subproblem_builder_time(subproblem::Model, node::Int64)
    @variable(subproblem, 0 <= x[i = 1:length(PPoints), t=1:T] <= 50, SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= W <= 100, SDDP.State, initial_value = 80)
    @variable(subproblem, y[i = 1:length(PPoints)-1, t=1:T] >= 0)
    @variable(subproblem, w[i = 1:length(PPoints)-1, t=1:T] >= 0)
    @variable(subproblem, b[i = 1:length(PPoints)-1, t=1:T] >= 0)
    @variable(subproblem, f >= 0)
    @variable(subproblem, s >= 0)
    
    @constraint(subproblem, clearing[i = 1:length(PPoints)-1, t=1:T], y[i,t] == 1* x[i,t].in  +  1* x[i+1,t].in)
    @constraint(subproblem, increasing[i = 1:length(PPoints)-1, t=1:T], x[i,t].out <= x[i+1,t].out)
    if node == 1
        @stageobjective(subproblem, 0)
        @constraint(subproblem, transfer, W.in == W.out)
    else
        @constraint(subproblem, production[i = 1:length(PPoints)-1, t=1:T], y[i,t] == w[i,t] + b[i,t])
        @constraint(subproblem, balance, W.out == W.in - sum(w[i,t] for i in 1:length(PPoints)-1 for t in 1:T) + f - s)
        @stageobjective(subproblem, 0)
        SDDP.parameterize(subproblem, Omega_t, P_t) do om
            # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
            # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
            
            # FIx the filling of the resource for every time step
            JuMP.fix(f, om.fill, force=true)

            # Define the Set of active constraints for every time step 
            I_t = Dict{Int64, Int64}(t => 0 for t in 1:T) 
            for t in 1:T
                for i in 1:length(PPoints)-1
                    if (om.price[t] >= PPoints[i]) && (om.price[t] <= PPoints[i+1])
                        I_t[t] = i
                    end
                end
            end
            # Set stageobjective, so only relevant values for every time step are included
            @stageobjective(subproblem, sum(om.price[t] * y[I_t[t],t] - 1.2 * om.price[t] b[I_t[t],t] for t in 1:T))
            # Fix / Deactivate Constraints by turning their coefficients to Zero or the correct value
            for t in 1:T
                for i in 1:length(PPoints)-1
                    if (i == I_t[t])
                        set_normalized_coefficient(clearing[i,t], y[i,t], 1)
                        set_normalized_coefficient(clearing[i,t], x[i,t].in, -((om.price[t] - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(clearing[i,t], x[i+1,t].in, -((PPoints[i+1] - om.price[t])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(balance, w[i,t], 1)
                        set_normalized_coefficient(production[i,t], w[i,t], -1)
                        set_normalized_coefficient(production[i,t], b[i,t], -1)
                        set_normalized_coefficient(production[i,t], y[i,t], 1)
                    else
                        set_normalized_coefficient(balance, w[i,t], 0)
                        set_normalized_coefficient(production[i,t], w[i,t], 0)
                        set_normalized_coefficient(production[i,t], b[i,t], 0)
                        set_normalized_coefficient(production[i,t], y[i,t], 0)
                        set_normalized_coefficient(clearing[i,t], y[i,t], 0)
                        set_normalized_coefficient(clearing[i,t], x[i,t].in, 0)
                        set_normalized_coefficient(clearing[i,t], x[i+1,t].in, 0)
                    end
                end
            end
        end
    end
    return
end

ErrorException: syntax: missing comma or ) in argument list

In [295]:
model2 = SDDP.LinearPolicyGraph(
    subproblem_builder_time;
    stages = 7,
    sense = :Max,
    upper_bound = 1e7,
    optimizer = CPLEX.Optimizer
)

SDDP.train(model2; iteration_limit=100)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 7
  state variables : 97
  scenarios       : 7.29000e+02
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [413, 413]
  VariableRef in MOI.LessThan{Float64}    : [98, 98]
  AffExpr in MOI.LessThan{Float64}        : [72, 72]
  VariableRef in MOI.GreaterThan{Float64} : [315, 316]
  AffExpr in MOI.EqualTo{Float64}         : [73, 145]
numerical stability report
  matrix range     [5e-01, 1e+00]
  objective range  [1e+00, 3e+01]
  bounds range     [5e+01, 1e+07]
  rhs range        [0e+00, 0e+00]
-------------------------------------------------------------------
 iteration    simulation      bound        time 


         1   4.516800e+04  1.383989e+05  8.999991e-02        26   1
         2   1.835040e+05  1.355280e+05  1.540000e-01        52   1


         3   1.835040e+05  1.355280e+05  2.300000e-01        78   1
         4   1.475220e+05  1.355280e+05  2.890000e-01       104   1


         5   1.235340e+05  1.355280e+05  3.520000e-01       130   1
         6   8.755200e+04  1.355280e+05  4.130001e-01       156   1


         7   1.235340e+05  1.355280e+05  4.760001e-01       182   1
         8   1.355280e+05  1.355280e+05  5.460000e-01       208   1


...


        95   1.115400e+05  1.355280e+05  8.029000e+00      2470   1


        96   1.835040e+05  1.355280e+05  8.194000e+00      2496   1
        97   8.755200e+04  1.355280e+05  8.290000e+00      2522   1


        98   1.115400e+05  1.355280e+05  8.379000e+00      2548   1


        99   1.835040e+05  1.355280e+05  8.493000e+00      2574   1
 

      100   1.115400e+05  1.355280e+05  8.606000e+00      2600   1
-------------------------------------------------------------------
status         : iteration_limit
total time (s) : 8.606000e+00
total solves   : 2600
best bound     :  1.355280e+05
simulation ci  :  1.354640e+05 ± 5.407126e+03
numeric issues : 0
-------------------------------------------------------------------



In [296]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model2,
    # The number of replications.
    100,
    # A list of names to record the values of.
    [:x, :y, :w, :W, :b, :f],
)

100-element Vector{Vector{Dict{Symbol, Any}}}:
 [Dict(:b => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], :w => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], :bellman_term => 135527.99999999983, :node_index => 1, :objective_state => nothing, :belief => Dict(1 => 1.0), :x => SDDP.State{Float64}[SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0)], :stage_objective => 0, :f => 0.0, :noise_term => nothing…), Dict(:b => [0.0 0.0 … 0.0 0.0; 50.0 50.0 … 50.0 50.0; 0.0 0.0 … 0.0 0.0], :w => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 

In [304]:
replication = 28
stage = 7
println("Bidding Curve Values: ", simulations[replication][stage][:x])
println("Inventory: ", simulations[replication][stage][:W])
println("Cleared Volume: ", simulations[replication][stage][:y])
println("Own Producion: ", simulations[replication][stage][:w])
println("Purchases: ", simulations[replication][stage][:b])
println("Filling in that stage: ", simulations[replication][stage][:f])

Bidding Curve Values: SDDP.State{Float64}[SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0); SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0

SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0)]
Inventory: SDDP.State{Float64}(100.0, 0.0)
Cleared Volume: [50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0 50.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
Own Producion: [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 15.0 50.0 50.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0